In [1]:
import pandas as pd

def assign_incrementality_weights(values, factors):
    # Step 1: create df to store initial values and further columns
    results = pd.DataFrame({'channel': [channel for channel in values.keys()], 'value': [value for value in values.values()]})

    # Step 2: calculate the total value volume
    total_value = 0
    for channel, value in values.items():
        total_value += value

    # Step 3: calculate the share of non-multiplied keys
    # Share for multiplied keys is irrelevant but an improvement can be made to null that
    non_multiplied_value = 0
    for channel, value in values.items():
        if channel not in factors.keys():
            non_multiplied_value += value
    if non_multiplied_value > 0:
        for channel, value in values.items():
            results['share'] = results['value'] / non_multiplied_value
    else:
        results["share"] = 0

    # Step 4: iterate through the values dictionary and multiply the values by the corresponding factor
    for i, row in results.iterrows():
        if row['channel'] in factors.keys():
            factor = factors.get(channel) # get the multiplying factor for the channel
            new_value = value * factor
            results.at[i, 'new_value'] = new_value
        else:
            results.at[i, 'new_value'] = results.at[i, 'value']

    # Step 5: calculate the surplus
    surplus = (results['new_value'].sum()) - total_value

    # Step 6: calculate the max possible subtraction for non-multiplied countries
    for i, row in results.iterrows():
        if row['channel'] not in factors.keys():
            abs_subtraction = row['share'] * surplus
            results.at[i, 'abs_subtraction'] = abs_subtraction
            max_subtraction = min(abs(abs_subtraction), row['value'])
            results.at[i, 'max_subtraction'] = max_subtraction
        else:
            results.at[i, 'max_subtraction'] = 0.0
            results.at[i, 'abs_subtraction'] = 0.0

    # Step 7: calculate the remaining surplus if the possible substraction is lower than reduciton required
    remaining_surplus = surplus - results['max_subtraction'].sum()

    # Step 8: distribute the remaining surplus proportionally among the multiplied countries
    if remaining_surplus > 0:
        for i, row in results.iterrows():
            if row['channel'] in factors.keys():
                proportion = row['value'] / results.loc[results['channel'].isin(factors.keys()), 'value'].sum()
                subtraction = proportion * remaining_surplus
                results.at[i, 'new_value'] -= subtraction

    # Step 9: calculate the final share of each channel
    for i, row in results.iterrows():
        if row['channel'] not in factors.keys():
            if results.at[i, 'max_subtraction'] == 0:
                results.at[i, 'weights'] = 1
            else:
                results.at[i, 'weights'] = (results.at[i, 'new_value'] - results.at[i, 'max_subtraction']) / results.at[i, 'value']
        else:
            if remaining_surplus > 0:
                results.at[i, 'weights'] = (results.at[i, 'new_value'] - results.at[i, 'max_subtraction']) / results.at[i, 'value']
            else:
                results.at[i, 'weights'] = factors.get(row['channel'])
    
    # Step 10: return results
    return results

In [2]:
starting_case = {'Google search': 60, 'Direct' : 30, 'GAC': 5, 'Meta': 5}
one_zero_case = {'Google search': 0, 'Direct' : 30, 'GAC': 5, 'Meta': 5}
two_zeros_case = {'Google search': 0, 'Direct' : 0, 'GAC': 5, 'Meta': 5}
factors = {'GAC': 2, 'Meta': 2}

In [3]:
assign_incrementality_weights(starting_case, factors)

,channel,value,share,new_value,abs_subtraction,max_subtraction,weights
0,Google search,60,0.666667,60.0,6.666667,6.666667,0.888889
1,Direct,30,0.333333,30.0,3.333333,3.333333,0.888889
2,GAC,5,0.055556,10.0,0.000000,0.000000,2.000000
3,Meta,5,0.055556,10.0,0.000000,0.000000,2.000000


In [4]:
assign_incrementality_weights(one_zero_case, factors)

,channel,value,share,new_value,abs_subtraction,max_subtraction,weights
0,Google search,0,0.000000,0.0,0.0,0.0,1.000000
1,Direct,30,1.000000,30.0,10.0,10.0,0.666667
2,GAC,5,0.166667,10.0,0.0,0.0,2.000000
3,Meta,5,0.166667,10.0,0.0,0.0,2.000000


In [5]:
assign_incrementality_weights(two_zeros_case, factors)

,channel,value,share,new_value,abs_subtraction,max_subtraction,weights
0,Google search,0,0,0.0,0.0,0.0,1.0
1,Direct,0,0,0.0,0.0,0.0,1.0
2,GAC,5,0,5.0,0.0,0.0,1.0
3,Meta,5,0,5.0,0.0,0.0,1.0


In [6]:
import pandas as pd
channel_volume_dictionary = {'Google search': 0, 'Direct' : 0, 'GAC': 5, 'Meta': 5}
factors = {'GAC': 2, 'Meta': 2}

In [7]:
# Step 1: create df to store initial values and further columns
results = pd.DataFrame({'channel': [channel for channel in channel_volume_dictionary.keys()], 'value': [value for value in channel_volume_dictionary.values()]})

In [8]:
# Step 2: calculate the total value volume
total_value = 0
for channel, value in channel_volume_dictionary.items():
    total_value += value

In [9]:
# Step 3: calculate the share of non-multiplied keys
# Share for multiplied keys is irrelevant but an improvement can be made to null that
non_multiplied_value = 0
for channel, value in channel_volume_dictionary.items():
    if channel not in factors.keys():
        non_multiplied_value += value
if non_multiplied_value > 0:
    for channel, value in channel_volume_dictionary.items():
        results['share'] = results['value'] / non_multiplied_value
else:
    results["share"] = 0

In [10]:
# Step 4: iterate through the values dictionary and multiply the values by the corresponding factor
for i, row in results.iterrows():
    if row['channel'] in factors.keys():
        factor = factors.get(channel) # get the multiplying factor for the channel, default is 1
        new_value = value * factor
        results.at[i, 'new_value'] = new_value
    else:
        results.at[i, 'new_value'] = results.at[i, 'value']

In [11]:
# Step 5: calculate the surplus
surplus = (results['new_value'].sum()) - total_value


In [12]:

# Step 6: calculate the max possible subtraction for non-multiplied countries
for i, row in results.iterrows():
    if row['channel'] not in factors.keys():
        abs_subtraction = row['share'] * surplus
        results.at[i, 'abs_subtraction'] = abs_subtraction
        max_subtraction = min(abs(abs_subtraction), row['value'])
        results.at[i, 'max_subtraction'] = max_subtraction
    else:
        results.at[i, 'max_subtraction'] = 0.0
        results.at[i, 'abs_subtraction'] = 0.0


In [13]:
# Step 7: calculate the remaining surplus if the possible substraction is lower than reduciton required
remaining_surplus = surplus - results['max_subtraction'].sum()

In [14]:
# Step 8: distribute the remaining surplus proportionally among the multiplied countries
if remaining_surplus > 0:
    for i, row in results.iterrows():
        if row['channel'] in factors.keys():
            proportion = row['value'] / results.loc[results['channel'].isin(factors.keys()), 'value'].sum()
            subtraction = proportion * remaining_surplus
            results.at[i, 'new_value'] -= subtraction

In [15]:
# Step 9: calculate the final share of each channel
for i, row in results.iterrows():
    if row['channel'] not in factors.keys():
        if results.at[i, 'max_subtraction'] == 0:
            results.at[i, 'weights'] = 1
        else:
            results.at[i, 'weights'] = (results.at[i, 'new_value'] - results.at[i, 'max_subtraction']) / results.at[i, 'value']
    else:
        if remaining_surplus > 0:
            results.at[i, 'weights'] = (results.at[i, 'new_value'] - results.at[i, 'max_subtraction']) / results.at[i, 'value']
        else:
            results.at[i, 'weights'] = factors.get(row['channel'])